In [745]:
import pandas as pd
import requests

In [746]:
df = pd.read_csv("https://raw.githubusercontent.com/HerbDavis20/HerbDavis20.github.io/refs/heads/main/project_files/gold_prices.csv")

df.head(10)

,USD/Gold per ounce,price,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,22/03/1968,40.00,NaN,NaN,NaN
1,25/03/1968,40.25,NaN,NaN,NaN
2,26/03/1968,41.10,NaN,NaN,NaN
3,27/03/1968,40.35,NaN,NaN,NaN
4,28/03/1968,39.60,NaN,NaN,NaN
5,29/03/1968,39.45,NaN,NaN,NaN
6,01/04/1968,38.60,NaN,NaN,NaN
7,03/04/1968,38.45,NaN,NaN,NaN
8,04/04/1968,38.05,NaN,NaN,NaN
9,05/04/1968,38.65,NaN,NaN,NaN


I am going to rename the columns to make it more clear of what they are.

In [747]:
df = df.rename(columns={
    'USD/Gold per ounce': 'date',
    'price': 'Price(USD$ / Ounce)'
})
df.head(10)

,date,Price(USD$ / Ounce),Unnamed: 2,Unnamed: 3,Unnamed: 4
0,22/03/1968,40.00,NaN,NaN,NaN
1,25/03/1968,40.25,NaN,NaN,NaN
2,26/03/1968,41.10,NaN,NaN,NaN
3,27/03/1968,40.35,NaN,NaN,NaN
4,28/03/1968,39.60,NaN,NaN,NaN
5,29/03/1968,39.45,NaN,NaN,NaN
6,01/04/1968,38.60,NaN,NaN,NaN
7,03/04/1968,38.45,NaN,NaN,NaN
8,04/04/1968,38.05,NaN,NaN,NaN
9,05/04/1968,38.65,NaN,NaN,NaN


next I want to remove the uncecessary columnns that are not relevant to me

In [748]:
df = df[['date', 'Price(USD$ / Ounce)']]
df.head(10)

,date,Price(USD$ / Ounce)
0,22/03/1968,40.00
1,25/03/1968,40.25
2,26/03/1968,41.10
3,27/03/1968,40.35
4,28/03/1968,39.60
5,29/03/1968,39.45
6,01/04/1968,38.60
7,03/04/1968,38.45
8,04/04/1968,38.05
9,05/04/1968,38.65


I also want to get the date formatted in a more standardised way so that it is easier to merge the differnt datasets down the line. I want to get the date in to YYYY-MM-DD format

In [749]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

df.head(10)

,date,Price(USD$ / Ounce)
0,1968-03-22,40.00
1,1968-03-25,40.25
2,1968-03-26,41.10
3,1968-03-27,40.35
4,1968-03-28,39.60
5,1968-03-29,39.45
6,1968-04-01,38.60
7,1968-04-03,38.45
8,1968-04-04,38.05
9,1968-04-05,38.65


Now I want to bring in the Bitcoin data so that i can begin analyising the correlation. 

In [750]:
#this is the url for the Bitcoin API data
#"https://eco-cors-proxy.netlify.app/proxy?url=https%3A%2F%2Fapi.stlouisfed.org%2Ffred%2Fseries%2Fobservations%3Fseries_id%3DCBBTCUSD%26api_key%3D22ee7a76e736e32f54f5df0a7171538d%26file_type%3Djson"

url = "https://eco-cors-proxy.netlify.app/proxy?url=https%3A%2F%2Fapi.stlouisfed.org%2Ffred%2Fseries%2Fobservations%3Fseries_id%3DCBBTCUSD%26api_key%3D22ee7a76e736e32f54f5df0a7171538d%26file_type%3Djson"

#this get the data out of the API url 
response = requests.get(url)
data = response.json()

#In the api, the data I want is under the heading of "observations" so i want to extract that and convert it to a dataframe 
df_observations = pd.DataFrame(data['observations'])

# Keep only the 'date' and 'value' columns
df1 = df_observations[['date', 'value']]
df1.head(30)

,date,value
0,2014-12-01,370
1,2014-12-02,378
2,2014-12-03,378
3,2014-12-04,377.1
4,2014-12-05,.
5,2014-12-06,378
6,2014-12-07,.
7,2014-12-08,375
8,2014-12-09,.
9,2014-12-10,360.5


I also want to alter the column names just to make it clearer which dataset this is

In [751]:
df1 = df1.rename(columns={
    'value': 'BTC_USD$'
})
df1.head(10)

,date,BTC_USD$
0,2014-12-01,370
1,2014-12-02,378
2,2014-12-03,378
3,2014-12-04,377.1
4,2014-12-05,.
5,2014-12-06,378
6,2014-12-07,.
7,2014-12-08,375
8,2014-12-09,.
9,2014-12-10,360.5


The data for the gold prices goes back in the past further than that of the Bitcoin and as I am going to be conducting a correlation analysis on them they need to have completely aligning data points. this means I need to filter the Gold data to remove vales that are not in the Bitcoin dataset. 

The earliest obervation in the Bitcoin data is 2014-12-01 so need to filter out all values less than this from the gold data. 

In [752]:
filtered_df = df[df['date'] >= '2014-12-01']
filtered_df.reset_index(drop=True, inplace=True) #this is to reset the index back to zero to help with merging later 
filtered_df.head(10)

,date,Price(USD$ / Ounce)
0,2014-12-01,1210.43
1,2014-12-02,1198.56
2,2014-12-03,1209.19
3,2014-12-04,1205.32
4,2014-12-05,1191.64
5,2014-12-08,1202.80
6,2014-12-09,1230.39
7,2014-12-10,1226.70
8,2014-12-11,1227.55
9,2014-12-12,1222.21


In the data for Bitcoin there are also lots of null values displaying "." instead of a value. I am going to have to remove these values from both dataframes as if set to 0 they would alter the correlation. 

In [753]:
df1 = df1[df1['BTC_USD$'] != "."]

df1.head(10) #displaying some of the dataframe to make sure that removing the null data has worked.

,date,BTC_USD$
0,2014-12-01,370
1,2014-12-02,378
2,2014-12-03,378
3,2014-12-04,377.1
5,2014-12-06,378
7,2014-12-08,375
9,2014-12-10,360.5
11,2014-12-12,350
17,2014-12-18,340
38,2015-01-08,288.99


Now I need to merge the two dataframes by their date as this is the column which is common between both dataframes

In [754]:
#thid sets the date as the common index for the merge 
filtered_df.set_index('date', inplace=True)
df1.set_index('date', inplace=True)


merged_df = pd.merge(filtered_df, df1, left_index=True, right_index=True, how='inner')

merged_df.reset_index(inplace=True)
merged_df.head(10)


,date,Price(USD$ / Ounce),BTC_USD$
0,2014-12-01,1210.43,370
1,2014-12-02,1198.56,378
2,2014-12-03,1209.19,378
3,2014-12-04,1205.32,377.1
4,2014-12-08,1202.80,375
5,2014-12-10,1226.70,360.5
6,2014-12-12,1222.21,350
7,2014-12-18,1197.65,340
8,2015-01-08,1208.63,288.99
9,2015-01-13,1230.36,260


After merging these dataframes and trying to perform the calculation for the 30 day rolling correlation I was running into an error which indicated some of the data was not in the correct format. instead of being an int or float the Bitcoin was an object which is not valid for use in calculations. To determin the data type of each set of data I ran the following line of code which told me the data type. 

In [755]:
print(merged_df.dtypes)

date                    object
Price(USD$ / Ounce)    float64
BTC_USD$                object
dtype: object


I could then run the following code to convert the data type into a number 

In [756]:
merged_df['BTC_USD$'] = pd.to_numeric(merged_df['BTC_USD$'], errors='coerce')
merged_df.head(31) #this is the data frame for the first line on the graph, where the calcualtion will be done from 

,date,Price(USD$ / Ounce),BTC_USD$
0,2014-12-01,1210.43,370.00
1,2014-12-02,1198.56,378.00
2,2014-12-03,1209.19,378.00
3,2014-12-04,1205.32,377.10
4,2014-12-08,1202.80,375.00
5,2014-12-10,1226.70,360.50
6,2014-12-12,1222.21,350.00
7,2014-12-18,1197.65,340.00
8,2015-01-08,1208.63,288.99
9,2015-01-13,1230.36,260.00


I can now run the checker again to see if the data is all in the correct format now. 

In [757]:
print(merged_df.dtypes) #both are outputting as a float which means I can begin the calculations.

date                    object
Price(USD$ / Ounce)    float64
BTC_USD$               float64
dtype: object


Now that the data is all in the corect format I can perform the calcualation. This calculates the PMCC value (r value), which is a method I have used to analyise correlation in the past. rolling(window=30) makes this a 30 day rolling correlation. 

In [758]:
# Calculate the 30-day rolling correlation between Gold Price and Bitcoin Price
rolling_corr = merged_df['Price(USD$ / Ounce)'].rolling(window=70).corr(merged_df['BTC_USD$'])



merged_df['Rolling_Corr_GOLD_BTC'] = rolling_corr


print(merged_df.head(30))


          date  Price(USD$ / Ounce)  BTC_USD$  Rolling_Corr_GOLD_BTC
0   2014-12-01              1210.43    370.00                    NaN
1   2014-12-02              1198.56    378.00                    NaN
2   2014-12-03              1209.19    378.00                    NaN
3   2014-12-04              1205.32    377.10                    NaN
4   2014-12-08              1202.80    375.00                    NaN
5   2014-12-10              1226.70    360.50                    NaN
6   2014-12-12              1222.21    350.00                    NaN
7   2014-12-18              1197.65    340.00                    NaN
8   2015-01-08              1208.63    288.99                    NaN
9   2015-01-13              1230.36    260.00                    NaN
10  2015-01-14              1229.55    120.00                    NaN
11  2015-01-15              1261.16    204.22                    NaN
12  2015-01-16              1279.81    199.46                    NaN
13  2015-01-19              1276.7

Now that I have the corrolation data between Bitcoin and Gold, I now want to do the same for Bitcoin and the VIX volatility indicator. This will make up the other line on my chart. After doing so I can merge the two together to have a dataframe with three columns of date, BTC/Gold correlation, and BTC/VIX correlation. 

I already have the Bitcoin data in the dataframe df1 so now I just need to get the VIX data which I am sourcing through the FRED API, as I did with BTC. 

**update** - as df1 has been merged earlier in the code it is not able to source it again to merge it with VIX. This means that I will load in and clean the Bitcoin data again under a differnt dataframe. 

In [759]:
#this is the url for the VIX API data
#"https://eco-cors-proxy.netlify.app/proxy?url=https%3A%2F%2Fapi.stlouisfed.org%2Ffred%2Fseries%2Fobservations%3Fseries_id%3DVIXCLS%26api_key%3D22ee7a76e736e32f54f5df0a7171538d%26file_type%3Djson"

url = "https://eco-cors-proxy.netlify.app/proxy?url=https%3A%2F%2Fapi.stlouisfed.org%2Ffred%2Fseries%2Fobservations%3Fseries_id%3DVIXCLS%26api_key%3D22ee7a76e736e32f54f5df0a7171538d%26file_type%3Djson"

#this get the data out of the API url 
response = requests.get(url)
data = response.json()

#In the api, the data I want is under the heading of "observations" so i want to extract that and convert it to a dataframe 
df_observations = pd.DataFrame(data['observations'])

# Keep only the 'date' and 'value' columns
df_VIX = df_observations[['date', 'value']]
df_VIX.head(10)

,date,value
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26
5,1990-01-09,22.20
6,1990-01-10,22.44
7,1990-01-11,20.05
8,1990-01-12,24.64
9,1990-01-15,26.34


I am going to do the same process that I did the Bitcoin data of: 
- renaming columns 
- filtering out values that dont match with the Bitcoin data
- converting values from object to float so can be used in calculating the PMCC


In [760]:
df_VIX = df_VIX.rename(columns={
    'value': 'VIX_value'
})

filtered_df_VIX = df_VIX[df_VIX['date'] >= '1990-01-02']
filtered_df_VIX.reset_index(drop=True, inplace=True) #this is to reset the index back to zero to help with merging later 
filtered_df_VIX['VIX_value'] = pd.to_numeric(filtered_df_VIX['VIX_value'], errors='coerce')

filtered_df_VIX.head(30)


,date,VIX_value
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26
5,1990-01-09,22.20
6,1990-01-10,22.44
7,1990-01-11,20.05
8,1990-01-12,24.64
9,1990-01-15,26.34


I also need to filter the VIX data to remove any void values. this way there are only numbers in the dataset which will allow the following calculations to be successful. 

In [761]:
filtered_df_VIX['VIX_value'] = pd.to_numeric(filtered_df_VIX['VIX_value'], errors='coerce')
filtered_df_VIX = filtered_df_VIX.dropna(subset=['VIX_value'])
filtered_df_VIX.reset_index(drop=True, inplace=True)


The code below is just repeating the process for loading in and cleaning the bitcoin data as per the reason in the **update** 

In [762]:
#this is the url for the Bitcoin API data
#"https://eco-cors-proxy.netlify.app/proxy?url=https%3A%2F%2Fapi.stlouisfed.org%2Ffred%2Fseries%2Fobservations%3Fseries_id%3DCBBTCUSD%26api_key%3D22ee7a76e736e32f54f5df0a7171538d%26file_type%3Djson"

url = "https://eco-cors-proxy.netlify.app/proxy?url=https%3A%2F%2Fapi.stlouisfed.org%2Ffred%2Fseries%2Fobservations%3Fseries_id%3DCBBTCUSD%26api_key%3D22ee7a76e736e32f54f5df0a7171538d%26file_type%3Djson"

#this get the data out of the API url 
response = requests.get(url)
data = response.json()

#In the api, the data I want is under the heading of "observations" so i want to extract that and convert it to a dataframe 
df_observations = pd.DataFrame(data['observations'])

# Keep only the 'date' and 'value' columns
df_BTC2 = df_observations[['date', 'value']]


df_BTC2 = df_BTC2.rename(columns={
    'value': 'BTC_USD$'
})


filtered_df_BTC2 = df_BTC2[df_BTC2['date'] >= '2014-12-01']
filtered_df_BTC2.reset_index(drop=True, inplace=True) #this is to reset the index back to zero to help with merging later 

filtered_df_BTC2 = filtered_df_BTC2[filtered_df_BTC2['BTC_USD$'] != "."]

filtered_df_BTC2.head(30) #displaying some of the dataframe to make sure that removing the null data has worked.

,date,BTC_USD$
0,2014-12-01,370
1,2014-12-02,378
2,2014-12-03,378
3,2014-12-04,377.1
5,2014-12-06,378
7,2014-12-08,375
9,2014-12-10,360.5
11,2014-12-12,350
17,2014-12-18,340
38,2015-01-08,288.99


Now I want to merge the bitcoin and VIX data into one dataframe. 

In [763]:
#thid sets the date as the common index for the merge 
filtered_df_VIX.set_index('date', inplace=True)
filtered_df_BTC2.set_index('date', inplace=True)

merged_df_VIX = pd.merge(filtered_df_VIX, filtered_df_BTC2, left_index=True, right_index=True, how='inner')


merged_df_VIX.reset_index(inplace=True) # I had to add in this line of code as it keeps the date heading in place, before this heaading was being removed which meant I could not merge the final dataframes at the end 

merged_df_VIX.head(31) #This is now the dataframe for one of the lines on my charts

,date,VIX_value,BTC_USD$
0,2014-12-01,14.29,370
1,2014-12-02,12.85,378
2,2014-12-03,12.47,378
3,2014-12-04,12.38,377.1
4,2014-12-08,14.21,375
5,2014-12-10,18.53,360.5
6,2014-12-12,21.08,350
7,2014-12-18,16.81,340
8,2015-01-08,17.01,288.99
9,2015-01-13,20.56,260


In [764]:
print(merged_df_VIX.dtypes)

date          object
VIX_value    float64
BTC_USD$      object
dtype: object


In [765]:
merged_df_VIX['BTC_USD$'] = pd.to_numeric(merged_df_VIX['BTC_USD$'], errors='coerce')

And finally I want to perform the calcualtion for BTC/ VIX correlation 

In [766]:
# Calculate the 30-day rolling correlation between Gold Price and Bitcoin Price
rolling_corr1 = merged_df_VIX['VIX_value'].rolling(window=70).corr(merged_df_VIX['BTC_USD$'])

#this adds the calculated corr value back into the origional dataframe
merged_df_VIX['Rolling_Corr_VIX_BTC'] = rolling_corr1


print(merged_df_VIX.head(90))


          date  VIX_value  BTC_USD$  Rolling_Corr_VIX_BTC
0   2014-12-01      14.29    370.00                   NaN
1   2014-12-02      12.85    378.00                   NaN
2   2014-12-03      12.47    378.00                   NaN
3   2014-12-04      12.38    377.10                   NaN
4   2014-12-08      14.21    375.00                   NaN
..         ...        ...       ...                   ...
85  2015-05-04      12.85    239.75             -0.017117
86  2015-05-05      14.31    236.76             -0.007918
87  2015-05-06      15.15    230.00             -0.015219
88  2015-05-07      15.13    237.99             -0.030496
89  2015-05-08      12.86    244.54             -0.010460

[90 rows x 4 columns]


In [767]:
print(merged_df.columns)

Index(['date', 'Price(USD$ / Ounce)', 'BTC_USD$', 'Rolling_Corr_GOLD_BTC'], dtype='object')


Now I want to merge on date the rolling Corr values from BTC/GOLD and BTC/VIX

In [768]:
#thid sets the date as the common index for the merge 
merged_df.set_index('date', inplace=True)
merged_df_VIX.set_index('date', inplace=True)

merged_df_FINAL = pd.merge(merged_df, merged_df_VIX, left_index=True, right_index=True, how='inner')
merged_df_FINAL.reset_index(inplace=True)
merged_df_FINAL['date'] = pd.to_datetime(merged_df_FINAL['date']).dt.strftime('%Y-%m-%d')
merged_df_FINAL.head(31) #This is now the dataframe for one of the lines on my charts

,date,Price(USD$ / Ounce),BTC_USD$_x,Rolling_Corr_GOLD_BTC,VIX_value,BTC_USD$_y,Rolling_Corr_VIX_BTC
0,2014-12-01,1210.43,370.00,NaN,14.29,370.00,NaN
1,2014-12-02,1198.56,378.00,NaN,12.85,378.00,NaN
2,2014-12-03,1209.19,378.00,NaN,12.47,378.00,NaN
3,2014-12-04,1205.32,377.10,NaN,12.38,377.10,NaN
4,2014-12-08,1202.80,375.00,NaN,14.21,375.00,NaN
5,2014-12-10,1226.70,360.50,NaN,18.53,360.50,NaN
6,2014-12-12,1222.21,350.00,NaN,21.08,350.00,NaN
7,2014-12-18,1197.65,340.00,NaN,16.81,340.00,NaN
8,2015-01-08,1208.63,288.99,NaN,17.01,288.99,NaN
9,2015-01-13,1230.36,260.00,NaN,20.56,260.00,NaN


Now that I have done all the data processing and the data is in the final dataframe of merged_df_FINAL i want to export it so I will download the code as a CSV file so that it can be used in a chart. 

In [769]:
merged_df_FINAL.to_csv("CorrolationBTC_GOLD_BTC_VIX_dataFINAL(7).csv", index=False)
df

,date,Price(USD$ / Ounce)
0,1968-03-22,40.00
1,1968-03-25,40.25
2,1968-03-26,41.10
3,1968-03-27,40.35
4,1968-03-28,39.60
...,...,...
14231,2024-09-20,2621.96
14232,2024-09-23,2628.40
14233,2024-09-24,2656.70
14234,2024-09-25,2656.82
